In [218]:
import numpy as np
import pandas as pd
import pickle
import datetime
import json

# RUN THESE

In [219]:
#load dictionary which maps a road segment to a bounding box
objID_to_bnd_box = pickle.load( open( "ID_to_bound_box_R2", "rb" ) )

In [220]:
#read in; had to use converters b/c the truck names had mixed types
df = pd.read_csv("20170314.csv", parse_dates=['date_fixed', 'date_fixed2'], converters={'truck_name': str})

In [221]:
df.count()

truck_name     65411
date_fixed     65411
date_fixed2    65411
address        65411
longitude      65411
latitude       65411
dtype: int64

In [222]:
df.head()

,truck_name,date_fixed,date_fixed2,address,longitude,latitude
0,178,2017-03-14 05:04:06,2017-03-14 05:04:06,201 THURBER ST & STEVENS PL ...,-76.135875,43.018057
1,263,2017-03-14 05:04:08,2017-03-14 05:04:08,45 ERIE BLVD W FROM NB WEST ST RAMP SYRACUSE 1...,-76.158347,43.051032
2,263,2017-03-14 05:04:12,2017-03-14 05:04:12,376 WATER ST W TO ERIE BLVD W ...,-76.158167,43.050835
3,266,2017-03-14 05:04:12,2017-03-14 05:04:13,1001-03 ERIE BLVD E & CROUSE AVE ...,-76.136336,43.050753
4,268,2017-03-14 05:04:13,2017-03-14 05:04:13,3120-22 GRANT BLVD ...,-76.160580,43.077202


In [223]:
#stripping white space from the truck names
df['truck_name'] = df['truck_name'].str.strip()
df['address'] =df['address'].str.strip()

In [224]:
#adjust time offset by 5 hours
#Not required for the March 13
#NOTE: March 14,15, and 16 are in DST, SO, 4 hour offset
df['date_fixed'] = df['date_fixed'] - pd.Timedelta(hours=4)

In [225]:
#sorted date/time
df = df.sort_values(by=['date_fixed'],na_position='first')

In [226]:
#reorder the index after sorting
df = df.reset_index(drop=True)

In [227]:
#get rid of data points from DPW site, by address. Gets rid of ~1300 points
df = df[~df.address.str.contains('1200 CANAL',na=False)]

In [181]:
df.count()

truck_name     64044
date_fixed     64044
date_fixed2    64044
address        64044
longitude      64044
latitude       64044
dtype: int64

In [228]:
#filter out points at DPW site, by long/lat
df = df[ (~df.longitude.between(-76.1136, -76.1059)) | (~df.latitude.between(43.0539, 43.0565)) ]

In [229]:
df.count()

truck_name     56141
date_fixed     56141
date_fixed2    56141
address        56141
longitude      56141
latitude       56141
dtype: int64

In [230]:
#keep only lon,lat
df = df[['date_fixed', 'longitude','latitude']]

In [231]:
#convert to numpy array
arr = df.values

In [232]:
#add a third column initialized to 'NaN'
arr = np.insert(arr, 3, None, axis=1)

In [233]:
#Add a column ('ID') which maps Long & Lat to a road segment (i.e 'OBJECTID' in the City Streets GeoJSON file) 
cnt = 0
for row in arr:
    lon = row[1]
    lat = row[2]
    for k,v in objID_to_bnd_box.items():
        if lon > min(v[0][0],v[1][0]) and lon < max(v[0][0],v[1][0]) and lat > min(v[0][1],v[1][1]) and lat < max(v[0][1],v[1][1]):
            arr[cnt][3] = k
    cnt += 1
           

In [234]:
#convert Numpy array to DataFrame to remove nulls
df = pd.DataFrame(arr)

In [235]:
#remove rows with nulls (i.e. data points which weren't captured by any bounding box)
df = df[df[3].notnull()]

In [236]:
#total count of mapped road segments
df.count()

0    50630
1    50630
2    50630
3    50630
dtype: int64

In [237]:
df = df.iloc[:,[0,3]]

In [238]:
#convert back to Numpy array to be stacked with data from other days in that date cluster
m14 = df.values

In [195]:
m14.shape

(50630, 2)

In [196]:
print (m13.shape)
print (m14.shape)
print (m15.shape)
print (m16.shape)

(4794, 2)
(50630, 2)
(53397, 2)
(35748, 2)


In [197]:
#stack the arrays by date
m13_16 = np.vstack((m13,m14,m15,m16))

In [198]:
#convert to DataFrame to sort by date
j = pd.DataFrame(m13_16,columns=['time','ID'])

In [199]:
# sort by date
j = j.sort_values(by=['time'])

## Dump time period final DF to pickle

In [200]:
pickle.dump( j, open( "mar_13_16_final_df", "wb" ) )

## Load j file (stacked and sorted DF)

In [970]:
j = pickle.load( open( "jan1_4_final_df", "rb" ) )

## Create main data file template 

In [201]:
#USE THIS
#create dictionary to reference for lapse times
lapse_time = {}
for i in range(1,5651):
    lapse_time[i] = []

In [202]:
j.count()

time    144569
ID      144569
dtype: int64

# Function to insert new lapse times into main data file

In [203]:
def insert_LT(ls,cnt):
    
    if cnt == 0:
        for k,v in lapse_time.items():
            if k in ls:
                lapse_time[k].append(0)
            else:
                lapse_time[k].append('null')
    else:
        for k,v in lapse_time.items():
            if k in ls:
                lapse_time[k].append(0)
            elif lapse_time[k][cnt - 1] == 'null':
                lapse_time[k].append('null')
            else:
                lapse_time[k].append(lapse_time[k][cnt - 1] + 2)
    
                
        
    
    

## Create temp dataframes by time period and call function to add to main data file

In [204]:
#NOTE!: If we want to put a null value at the first index, we will need to have 49 values in the list of elapsed times
# (1 for each period and the first one being null). Would need to initialize cnt to 1
#ANOTHER NOTE: the range for 'day' needs to be adjusted for each dataset
cnt = 0
for day in range(13,17):
    for hour in range(2,26,2):
        temp_df = j[ (j.time.dt.day == day) & (j.time.dt.hour < hour) & (j.time.dt.hour >= hour - 2)]
        tlist = temp_df.ID.unique()
        insert_LT(tlist,cnt)
        cnt += 1
        

In [205]:
#convert lapse_times dict into a json array
LT = []
for i in range(1,5651):
    d = {"ID":str(i), "LT": lapse_time[i]}
    LT.append(d)

In [206]:
print (LT[478])

{'ID': '479', 'LT': ['null', 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 0, 2, 4, 6, 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 0, 2, 4, 0, 2, 4, 6, 8, 10, 0, 2, 4, 6, 8, 10, 12]}


# GeoJson File for comparing lapse times to a given road

In [209]:
with open('City_Streets_2011.geojson') as f_in:
    roads = json.load(f_in)

In [215]:
#dictionary mapping road ID to it's properties
ID_to_street = {}
for each in roads['features']:
    ID_to_street[each['properties']['OBJECTID']] = each

# Dump final data file

In [207]:
#dump final data file
with open("LT_M13_16.json", "w") as text_file:
    json.dump(LT, text_file)

# Testing

In [663]:
j1_4[90000][0].hour

14

In [712]:
#convert to DataFrame to sort by date
j = pd.DataFrame(j1_4,columns=['time','ID'])

In [713]:
j = j.sort_values(by=['time'])

In [714]:
j1_4 = j.values

In [748]:
j.loc[1000,][0]

Timestamp('2018-01-01 01:13:28')

In [751]:
cnt =0
for row in j1_4:
    if row[0].day == 1 & row[0].hour < 12:
        cnt+=1 
print (cnt)

15101


In [752]:
ts = j[j.time.dt.hour == 3]

In [753]:
ts.count()

time    3332
ID      3332
dtype: int64

In [785]:
lapse_time[1][1-0]

'null'

In [169]:
t_df = df[ df.date_fixed.dt.hour == 4]

In [170]:
t_df.head()

,truck_name,date_fixed,date_fixed2,address,longitude,latitude
16,263,2017-03-14 04:53:54,2017-03-14 04:53:54,415 FRANKLIN ST N ...,-76.154851,43.054566
102,284,2017-03-14 04:53:55,2017-03-14 04:53:56,764 COMSTOCK AVE & EUCLID AVE ...,-76.129813,43.036453
103,179,2017-03-14 04:53:55,2017-03-14 04:53:55,1820 EB I 690 SYRACUSE 13210 ...,-76.111369,43.055477
104,263,2017-03-14 04:53:58,2017-03-14 04:53:58,415 FRANKLIN ST N ...,-76.155033,43.054668
105,261,2017-03-14 04:53:59,2017-03-14 04:53:59,1073-85 GENESEE ST E & UNIVERSITY ...,-76.134912,43.046925


In [83]:
tf = df[ (df.date_fixed.dt.hour != df.date_fixed2.dt.hour) | (df.date_fixed.dt.minute != df.date_fixed2.dt.minute)]

In [84]:
tf.count()

truck_name     511
date_fixed     511
date_fixed2    511
address        511
longitude      511
latitude       511
dtype: int64

In [34]:
tf = tf.sort_values(by=['truck_name'])

In [85]:
tf.date_fixed2.unique().size

319

## Using Manhattan distance to filter close points. Changed approach to problem, so not used.

In [ ]:
# get the manhattan distance between consecutive data points
#df['manh_dist'] = abs(df.longitude - df.longitude.shift()) + abs(df.latitude - df.latitude.shift())

In [ ]:
#filter out close data points
#df = df[df.manh_dist > .0005]

In [ ]:
#filter out unreasonably far data points
#df = df[df.manh_dist < .01]

## More testing

In [ ]:
#How to get counts of unique values for how many differnt days (there is only 1 day here, i.e. Jan 1)
df['date_fixed'].dt.day.value_counts()

In [ ]:
#test iterrows; delete eventually
tst2 = []
for index, row in df.iterrows():
    tst2.append([index,row])
    

In [ ]:
# an index and row
tst2[10]

In [ ]:
# a row (as a Series)
tst2[10][1]

In [ ]:
tst2[1000][1].latitude

In [ ]:
objID_to_bnd_box[10]

# Below here is not used

In [ ]:
#NOT USED; generate test dataframe
test_row_iter = pd.DataFrame(np.random.randn(20,3),columns=list('ABC'))

In [ ]:
#NOT USED
#for testing: set column C to absolute values
test_row_iter['C'] = abs(test_row_iter['C'])

In [ ]:
#NOT USED
#how to iterate over rows and set a boolean value based on a conditional
for i in range(1,len(test_row_iter)):
    if test_row_iter.loc[i, 'C'] > 0:
        test_row_iter.loc[i,'D'] = True

In [ ]:
#NOT USED
#set first boolean of the 'Keep' column to 'True', since we always want to keep the first data point
test_row_iter.loc[0,'Keep'] = True
#create a boolean column based on an the accumulated value of a column
threshhold = 2.
cum_dist = 0.
for i in range(1,len(test_row_iter)):
    if test_row_iter.loc[i, 'C'] + cum_dist > threshhold:
        test_row_iter.loc[i,'Keep'] = True
        cum_dist = 0
    else:
        test_row_iter.loc[i,'Keep'] = False
        cum_dist += test_row_iter.loc[i, 'C']
            